In [ ]:
from datetime import datetime
from getpass import getpass

rdm_url = 'https://bh.rdm.yzwlab.com'
idp_name_1 = None # 'GakuNin RDM IdP'
idp_username_1 = None
idp_password_1 = None
rdm_project_name = 'TEST-METADATA-{}'.format(datetime.now().strftime('%Y%m%d-%H%M%S'))
default_result_path = None
close_on_fail = False
transition_timeout = 10000

In [ ]:
if idp_username_1 is None:
    idp_username_1 = input(prompt=f'Username for {idp_name_1}')
if idp_password_1 is None:
    idp_password_1 = getpass(prompt=f'Password for {idp_username_1}@{idp_name_1}')
(len(idp_username_1), len(idp_password_1))

In [ ]:
import tempfile

work_dir = tempfile.mkdtemp()
if default_result_path is None:
    default_result_path = work_dir
work_dir

# KAKEN情報を用いた自動補完

- サブシステム名: アドオン
- ページ/アドオン: Metadata
- 機能分類: メタデータ入力
- シナリオ名: KAKEN情報による自動補完-自分のe-Rad研究者番号を用いた補完
- 用意するテストデータ: URL一覧、アカウント1(既存ユーザー1: GRDM)
- 事前条件: KAKENデータが更新済みであること

## ウェブブラウザの新規プライベートウィンドウでGRDMトップページを表示する

GRDMトップページが表示されること

In [ ]:
import importlib
import pandas as pd

import scripts.playwright
importlib.reload(scripts.playwright)

from scripts.playwright import *
from scripts import grdm

await init_pw_context(close_on_fail=False, last_path=default_result_path)

In [ ]:
import time

async def _step(page):
    await page.goto(rdm_url)

    # 同意する をクリック
    await page.locator('//button[text() = "同意する"]').click()

    # 同意する が表示されなくなったことを確認
    await expect(page.locator('//button[text() = "同意する"]')).to_have_count(0, timeout=500)

await run_pw(_step)

## アカウント1 ログイン情報を用いてGakuNin RDMにログインする

(IdPに関するログイン情報が与えられた場合、)
GakuNin Embeded DSのプルダウンを展開し、IdPリストから指定されたIdPを選択する。その後、アカウントのID/Passwordを入力して「Login」ボタンを押下する。

(IdPが指定されていない場合、)
CASのログイン操作を実施する。

In [ ]:
import scripts.grdm
importlib.reload(scripts.grdm)

async def _step(page):
    await scripts.grdm.login(
        page, idp_name_1, idp_username_1, idp_password_1, transition_timeout=transition_timeout
    )

    # GRDMのボタンが表示されることを確認
    await expect(page.locator('//*[text() = "プロジェクト管理者"]')).to_be_visible(timeout=transition_timeout)

await run_pw(_step)

## ログインユーザー表示をクリックする

ドロップダウンに「設定」が現れること

In [ ]:
async def _step(page):
    await page.locator(f'//a[@data-test-auth-dropdown-toggle]').click()
    await expect(page.locator(f'//a[@data-test-ad-settings]')).to_be_visible(timeout=1000)

await run_pw(_step)

## 「設定」をクリックする

「プロフィール」画面が現れること

In [ ]:
async def _step(page):
    await page.locator(f'//a[@data-test-ad-settings]').click()
    await expect(page.locator(f'//a[text() = "ID"]')).to_be_visible(timeout=1000)

await run_pw(_step)

## 「ID」をクリックする

「e-Rad研究者番号」画面が現れること

In [ ]:
async def _step(page):
    await page.locator(f'//a[text() = "ID"]').click()
    await expect(page.locator(f'//input[@data-bind = "value: erad"]')).to_be_editable(timeout=1000)

await run_pw(_step)

## 「あなたのe-Rad番号」に「50735975」を入力する

「e-Rad」の項目が設定されること

In [ ]:
async def _step(page):
    await page.locator(f'//input[@data-bind = "value: erad"]').fill('50735975')
    await expect(page.locator(f'//button[text() = "保存" and ancestor::div[contains(@class,"active")]]')).to_be_visible(timeout=1000)

await run_pw(_step)

## 「保存」をクリックする

「設定が更新されました」と緑色で表示されること

In [ ]:
async def _step(page):
    await page.locator(f'//button[text() = "保存" and ancestor::div[contains(@class,"active")]]').click()
    await expect(page.locator(f'//p[contains(@class, "text-success") and contains(text(), "設定が更新されました")]')).to_be_visible(timeout=1000)

await run_pw(_step)

## ダッシュボードのURLを開き、プロジェクト一覧に指定されたタイトルのプロジェクトがない場合、指定された名前のプロジェクトを作成する

プロジェクト一覧に当該プロジェクト名が表示されていない場合、「新規プロジェクト作成」をクリックし、その名前を入力、「作成」をクリックする。

In [ ]:
from urllib.parse import urljoin
import scripts.grdm
importlib.reload(scripts.grdm)

async def _step(page):
    await page.goto(urljoin(rdm_url, 'dashboard'))
    await expect(page.locator('//*[@data-test-create-project-modal-button]')).to_have_count(1)

    await scripts.grdm.ensure_project_exists(page, rdm_project_name, transition_timeout=transition_timeout)
        
await run_pw(_step)

## ダッシュボードのプロジェクト一覧から作成したプロジェクトをクリックする

作成したプロジェクトのプロジェクトダッシュボードが表示されること

In [ ]:
async def _step(page):
    await page.locator(f'//*[@data-test-dashboard-item-title and text() = "{rdm_project_name}"]').click()
    
    await expect(page.locator('//a[text() = "アドオン"]')).to_be_visible(timeout=10000)
    await expect(page.locator('//*[contains(@class, "title-text")]//*[text() = "NII Storage"]')).to_be_visible(timeout=10000)
    time.sleep(1)

await run_pw(_step)

## プロジェクトダッシュボードの上部メニューから「メタデータ」をクリックする

メタデータの一覧ページが表示されること

In [ ]:
async def _step(page):
    await page.locator(f'//a[contains(text(), "メタデータ")]').click()
    await expect(page.locator('//*[@data-test-new-metadata-button]')).to_be_visible(timeout=10000)

await run_pw(_step)

## 「新規メタデータ」をクリックする

スキーマ選択ダイアログが表示されること

In [ ]:
async def _step(page):
    await page.locator('//*[@data-test-new-metadata-button]').click()

    await expect(page.locator('//*[@data-test-new-report-modal-schema="公的資金による研究データのメタデータ登録"]')).to_be_visible(timeout=transition_timeout)
    await expect(page.locator('//*[@data-test-new-report-modal-create-report-button]')).to_be_visible(timeout=10000)
    time.sleep(1)

await run_pw(_step)

## 「メタデータを作成」をクリックする

メタデータ編集ウィンドウが表示されること

In [ ]:
async def _step(page):
    await page.locator('//*[@data-test-new-report-modal-create-report-button]').click()
    
    await expect(page.locator('//*[contains(text(), "資金配分機関情報")]/../following-sibling::div[1]//*[contains(@class, "ember-power-select-status-icon")]')).to_be_attached(timeout=transition_timeout)
    await expect(page.locator('//*[contains(text(), "体系的番号におけるプログラム情報コード")]/../following-sibling::div[1]//input')).to_be_editable(timeout=1000)
    await expect(page.locator('//*[contains(text(), "プログラム名 (日本語)")]/../following-sibling::div[1]//input')).to_be_editable(timeout=1000)
    await expect(page.locator('//*[contains(text(), "Program name (English)")]/../following-sibling::div[1]//input')).to_be_editable(timeout=1000)
    await expect(page.locator('//*[contains(text(), "体系的番号")]/../following-sibling::div[1]//*[contains(@class, "ember-power-select-status-icon")]')).to_be_attached(timeout=1000)
    await expect(page.locator('//*[contains(text(), "プロジェクト名 (日本語)")]/../following-sibling::div[1]//input')).to_be_editable(timeout=1000)
    await expect(page.locator('//*[contains(text(), "Project name (English)")]/../following-sibling::div[1]//input')).to_be_editable(timeout=1000)
    await expect(page.locator('//*[contains(text(), "プロジェクトの分野")]/../following-sibling::div[1]//*[contains(@class, "ember-power-select-status-icon")]')).to_be_attached(timeout=1000)

await run_pw(_step)

## 「体系的番号」をクリックする

ドロップダウン中に「独立行政法人日本学術振興会」が現れること

In [ ]:
async def _step(page):
    await page.locator('//*[contains(text(), "体系的番号")]/../following-sibling::div[1]//*[contains(@class, "ember-power-select-status-icon")]').click()
    
    await expect(page.locator('//li[contains(@class, "ember-power-select-option")]')).to_be_visible(timeout=transition_timeout)

await run_pw(_step)

## 最初の候補をクリックする

資金配分機関情報, 体系的番号におけるプログラム情報コード, 体系的番号, プログラム名(日本語), プロジェクト名(日本語), プロジェクトの分野が入力されること

In [ ]:
async def _step(page):
    # Use . instead of text() to match text in descendant elements as well
    await page.locator('//li[contains(@class, "ember-power-select-option") and contains(., "リアルワールドデータ")]').click()
    
    await expect(page.locator('//*[contains(text(), "資金配分機関情報")]/../following-sibling::div[1]//*[contains(@class, "ember-power-select-selected-item")]')).to_have_text('JSPS', timeout=transition_timeout)
    await expect(page.locator('//*[contains(text(), "体系的番号におけるプログラム情報コード")]/../following-sibling::div[1]//input')).to_have_value('', timeout=1000)
    await expect(page.locator('//*[contains(text(), "体系的番号")]/../following-sibling::div[1]//*[contains(@class, "ember-power-select-selected-item")]')).not_to_have_text('', timeout=1000)
    await expect(page.locator('//*[contains(text(), "プログラム名 (日本語)")]/../following-sibling::div[1]//input')).to_have_value('科学研究費助成事業', timeout=1000)
    await expect(page.locator('//*[contains(text(), "Program name (English)")]/../following-sibling::div[1]//input')).to_have_value('Grants-in-Aid for Scientific Research', timeout=1000)
    await expect(page.locator('//*[contains(text(), "プロジェクト名 (日本語)")]/../following-sibling::div[1]//input')).not_to_have_value('', timeout=1000)
    await expect(page.locator('//*[contains(text(), "Project name (English)")]/../following-sibling::div[1]//input')).not_to_have_value('', timeout=1000)
    await expect(page.locator('//*[contains(text(), "プロジェクトの分野")]/../following-sibling::div[1]//*[contains(@class, "ember-power-select-selected-item")]')).to_have_text('189', timeout=1000)

await run_pw(_step)

## ダッシュボードのURLを開く

GRDMダッシュボードが表示されること

In [ ]:
async def _step(page):
    await page.goto(urljoin(rdm_url, 'dashboard'))
    await expect(page.locator(f'//*[@data-test-dashboard-item-title and text() = "{rdm_project_name}"]')).to_be_visible(timeout=30000)
    time.sleep(1)

await run_pw(_step)

## ダッシュボードのプロジェクト一覧から作成したプロジェクトをクリックする

作成したプロジェクトのプロジェクトダッシュボードが表示されること

In [ ]:
async def _step(page):
    await page.locator(f'//*[@data-test-dashboard-item-title and text() = "{rdm_project_name}"]').click()
    
    await expect(page.locator('//a[text() = "アドオン"]')).to_be_visible(timeout=10000)
    await expect(page.locator('//*[contains(@class, "title-text")]//*[text() = "NII Storage"]')).to_be_visible(timeout=10000)
    time.sleep(1)

await run_pw(_step)

## 「NII Storage」をクリックする

「メタデータ編集」ボタンが表示されること (表示まで数秒かかる可能性がある)

In [ ]:
async def _step(page):
    target_storage_name = 'NII Storage'
    await grdm.get_select_expanded_storage_title_locator(page, target_storage_name).click()
    
    await expect(page.locator('//*[text() = "メタデータ編集"]')).to_be_enabled(timeout=10000)

await run_pw(_step)

## 「新規フォルダ作成」をクリックする

フォルダ名入力テキストフィールドが表示されること

In [ ]:
async def _step(page):
    await page.locator('//*[text() = "新規フォルダ作成"]').click()
    
    await expect(page.locator('//input[@id = "createFolderInput"]')).to_be_editable(timeout=1000)

await run_pw(_step)

## 「KAKENメタデータ(ENTER)」を入力する

KAKENメタデータフォルダが作成されること

In [ ]:
async def _step(page):
    await page.locator('//input[@id = "createFolderInput"]').fill('KAKENメタデータ')
    await page.keyboard.press('Enter')

    await expect(page.locator('//*[text() = "KAKENメタデータ"]')).to_be_visible(timeout=10000)

await run_pw(_step)

## 「KAKENメタデータ」フォルダをクリックする

「メタデータ編集」ボタンが表示されること (表示まで数秒かかる可能性がある)

In [ ]:
async def _step(page):
    await page.locator('//*[text() = "KAKENメタデータ"]').click()

    await expect(page.locator('//*[text() = "メタデータ編集"]')).to_be_enabled(timeout=1000)

await run_pw(_step)

##  「メタデータ編集」をクリックする

「メタデータ編集」ダイアログが表示されること

In [ ]:
async def _step(page):
    await page.locator('//*[text() = "メタデータ編集"]').click()

    await expect(page.locator('//label[contains(text(), "メタデータ様式")]/following-sibling::select')).to_be_editable(timeout=1000)
    time.sleep(1)

await run_pw(_step)

##  「メタデータ様式」に「公的資金による研究データのメタデータ登録」を入力する

「データNo.」フィールドが表示されること

In [ ]:
async def _step(page):
    await page.locator('//label[contains(text(), "メタデータ様式")]/following-sibling::select').select_option('公的資金による研究データのメタデータ登録')

    await expect(page.locator('//label[contains(text(), "データ No.")]/../following-sibling::div//input')).to_be_editable(timeout=1000)

await run_pw(_step)

##  「データ作成者」の「追加」をクリックする

作成者入力フィールドが表示されること

In [ ]:
async def _step(page):
    await page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//*[text() = "追加"]').click()

    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//tbody/tr[1]/td[1]//input')).to_be_editable(timeout=10000)

await run_pw(_step)

##  「e-Rad 研究者番号」に「5073」を入力する

Yusuke Komiyama が表示されること

In [ ]:
async def _step(page):
    locator = page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div[1]//tbody/tr[1]/td[1]//input')
    value = '5073'
    await locator.fill(value)

    await expect(page.locator('//*[contains(text(), "Yusuke Komiyama")]')).to_have_count(1, timeout=transition_timeout)

await run_pw(_step)

## 「Yusuke Komiyama」をクリックする

「e-Rad研究者番号」「名前 (日本語)」, 「Name (English)」が入力されること


In [ ]:
import re

async def _step(page):
    await page.locator('(//*[contains(text(), "Yusuke Komiyama")])[1]').click()
    
    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div[1]//tbody/tr[1]/td[1]//input')).to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//tbody/tr[1]/td[2]//input')).to_have_value(re.compile('.+'))
    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//tbody/tr[1]/td[3]//input')).to_have_value(re.compile('.+'))

await run_pw(_step)

## データ作成者の×をクリックする

データはありません と表示されること

In [ ]:
import asyncio

async def _step(page):
    await page.locator('//*[contains(@class, "remove-row")]//i[contains(@class, "fa-times")]').click()
    await expect(page.locator('//*[text() = "データはありません"]')).to_be_visible(timeout=1000)

await run_pw(_step)

##  「データ作成者」の「追加」をクリックする

作成者入力フィールドが表示されること

In [ ]:
async def _step(page):
    await page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//*[text() = "追加"]').click()

    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//tbody/tr[1]/td[1]//input')).to_be_editable(timeout=10000)

await run_pw(_step)

##  「名前(日本語)」に「悠介」を入力する

Yusuke Komiyama が表示されること

In [ ]:
async def _step(page):
    locator = page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div[1]//tbody/tr[1]/td[2]//input')
    value = '悠介'
    await locator.fill(value)

    await expect(page.locator('//*[contains(text(), "50735975")]')).to_have_count(1, timeout=transition_timeout)

await run_pw(_step)

## 「Yusuke Komiyama」をクリックする

「e-Rad研究者番号」「名前 (日本語)」, 「Name (English)」が入力されること


In [ ]:
import re

async def _step(page):
    await page.locator('(//*[contains(text(), "50735975")])[1]').click()
    
    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div[1]//tbody/tr[1]/td[1]//input')).to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//tbody/tr[1]/td[2]//input')).to_have_value(re.compile('.+'))
    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//tbody/tr[1]/td[3]//input')).to_have_value(re.compile('.+'))

await run_pw(_step)

## データ作成者の×をクリックする

データはありません と表示されること

In [ ]:
import asyncio

async def _step(page):
    await page.locator('//*[contains(@class, "remove-row")]//i[contains(@class, "fa-times")]').click()
    await expect(page.locator('//*[text() = "データはありません"]')).to_be_visible(timeout=1000)

await run_pw(_step)

##  「データ作成者」の「追加」をクリックする

作成者入力フィールドが表示されること

In [ ]:
async def _step(page):
    await page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//*[text() = "追加"]').click()

    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//tbody/tr[1]/td[1]//input')).to_be_editable(timeout=10000)

await run_pw(_step)

##  「Name (English)」に「Yusuke」を入力する

Yusuke Komiyama が表示されること

In [ ]:
async def _step(page):
    locator = page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div[1]//tbody/tr[1]/td[3]//input')
    value = 'Yusuke'
    await locator.fill(value)

    await expect(page.locator('//*[contains(text(), "50735975")]')).to_have_count(1, timeout=transition_timeout)

await run_pw(_step)

## 「Yusuke Komiyama」をクリックする

「e-Rad研究者番号」「名前 (日本語)」, 「Name (English)」が入力されること


In [ ]:
import re

async def _step(page):
    await page.locator('(//*[contains(text(), "50735975")])[1]').click()
    
    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div[1]//tbody/tr[1]/td[1]//input')).to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//tbody/tr[1]/td[2]//input')).to_have_value(re.compile('.+'))
    await expect(page.locator('//label[contains(text(), "データ作成者")]/../following-sibling::div//tbody/tr[1]/td[3]//input')).to_have_value(re.compile('.+'))

await run_pw(_step)

## 「データ管理者の種類」にて「個人」を選択する

すでに値が表示されている場合は選択不要。「個人」が表示されること

In [ ]:
async def _step(page):
    locator = page.locator('//label[contains(text(), "データ管理者の種類")]/../following-sibling::select[1]')
    value = '個人'
    await locator.select_option(value)
    await page.keyboard.press('Tab')

    await expect(locator).to_have_value('individual', timeout=1000)

await run_pw(_step)

##  「データ管理者の e-Rad 研究者番号」に「5073」を入力する

候補 Yusuke Komiyama が表示されること

In [ ]:
async def _step(page):
    locator = page.locator('//label[contains(text(), "データ管理者の e-Rad 研究者番号")]/../following-sibling::div[1]//input')
    await locator.fill('5073')

    await expect(page.locator('//*[contains(text(), "Yusuke Komiyama")]')).to_have_count(1, timeout=10000)

await run_pw(_step)

## 「KAKEN」側候補をクリックする

「データ管理者 (日本語)」「Data manager (English)」「データ管理者の所属組織名 (日本語)」が入力されること。 「Contact organization of data manager (English)」は空欄であること


In [ ]:
async def _step(page):
    await page.locator('(//*[contains(text(), "Yusuke Komiyama")])[1]').click()
    
    await expect(page.locator('//label[contains(text(), "データ管理者の所属組織名 (日本語)")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "Contact organization of data manager (English)")]/../following-sibling::div[1]//input')).not_to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "データ管理機関コード")]/../following-sibling::div[1]//input')).to_have_value('')
    await expect(page.locator('//label[contains(text(), "データ管理者 (日本語)")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'))
    await expect(page.locator('//label[contains(text(), "Data manager (English")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'))

await run_pw(_step)

##  「データ管理者の e-Rad 研究者番号」「データ管理者 (日本語)」「Data manager (English)」「データ管理者の所属組織名 (日本語)」を空欄とし、「データ管理者 (日本語)」に「悠介」を入力する

候補 Yusuke Komiyama が表示されること

In [ ]:
async def _step(page):
    await page.locator('//label[contains(text(), "データ管理者の e-Rad 研究者番号")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "データ管理者の所属組織名 (日本語)")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "Contact organization of data manager (English)")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "データ管理者 (日本語)")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "Data manager (English")]/../following-sibling::div[1]//input').fill('')

    locator = page.locator('//label[contains(text(), "データ管理者 (日本語)")]/../following-sibling::div[1]//input')
    await locator.fill('悠介')

    await expect(page.locator('//*[contains(text(), "50735975")]')).to_have_count(1, timeout=10000)

await run_pw(_step)

## 「KAKEN」側候補をクリックする

「データ管理者の e-Rad 研究者番号」「データ管理者 (日本語)」「Data manager (English)」「データ管理者の所属組織名 (日本語)」が入力されること。 「Contact organization of data manager (English)」は空欄であること


In [ ]:
async def _step(page):
    await page.locator('(//*[contains(text(), "50735975")])[1]').click()
    
    await expect(page.locator('//label[contains(text(), "データ管理者の e-Rad 研究者番号")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "データ管理者の所属組織名 (日本語)")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "Contact organization of data manager (English)")]/../following-sibling::div[1]//input')).not_to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "データ管理機関コード")]/../following-sibling::div[1]//input')).to_have_value('')
    await expect(page.locator('//label[contains(text(), "データ管理者 (日本語)")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'))
    await expect(page.locator('//label[contains(text(), "Data manager (English")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'))

await run_pw(_step)

##  「データ管理者の e-Rad 研究者番号」「データ管理者 (日本語)」「Data manager (English)」「データ管理者の所属組織名 (日本語)」を空欄とし、「Data manager (English)」に「Yusuke」を入力する

候補 Yusuke Komiyama が表示されること

In [ ]:
async def _step(page):
    await page.locator('//label[contains(text(), "データ管理者の e-Rad 研究者番号")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "データ管理者の所属組織名 (日本語)")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "Contact organization of data manager (English)")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "データ管理者 (日本語)")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "Data manager (English")]/../following-sibling::div[1]//input').fill('')

    locator = page.locator('//label[contains(text(), "Data manager (English")]/../following-sibling::div[1]//input')
    await locator.fill('Yusuke')

    await expect(page.locator('//*[contains(text(), "50735975")]')).to_have_count(1, timeout=10000)

await run_pw(_step)

## 「KAKEN」側候補をクリックする

「データ管理者の e-Rad 研究者番号」「データ管理者 (日本語)」「Data manager (English)」「データ管理者の所属組織名 (日本語)」が入力されること。 「Contact organization of data manager (English)」は空欄であること


In [ ]:
async def _step(page):
    await page.locator('(//*[contains(text(), "50735975")])[1]').click()
    
    await expect(page.locator('//label[contains(text(), "データ管理者の e-Rad 研究者番号")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "データ管理者の所属組織名 (日本語)")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "Contact organization of data manager (English)")]/../following-sibling::div[1]//input')).not_to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "データ管理機関コード")]/../following-sibling::div[1]//input')).to_have_value('')
    await expect(page.locator('//label[contains(text(), "データ管理者 (日本語)")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'))
    await expect(page.locator('//label[contains(text(), "Data manager (English")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'))

await run_pw(_step)

##  「データ管理者の e-Rad 研究者番号」「データ管理者 (日本語)」「Data manager (English)」「データ管理者の所属組織名 (日本語)」を空欄とし、「データ管理者の所属組織名 (日本語)」に「国立情報学」を入力する

候補 Yusuke Komiyama が表示されること

In [ ]:
async def _step(page):
    await page.locator('//label[contains(text(), "データ管理者の e-Rad 研究者番号")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "データ管理者の所属組織名 (日本語)")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "Contact organization of data manager (English)")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "データ管理者 (日本語)")]/../following-sibling::div[1]//input').fill('')
    await page.locator('//label[contains(text(), "Data manager (English")]/../following-sibling::div[1]//input').fill('')

    locator = page.locator('//label[contains(text(), "データ管理者の所属組織名 (日本語)")]/../following-sibling::div[1]//input')
    await locator.fill('国立情報学')

    await expect(page.locator('//*[contains(text(), "50735975")]')).to_have_count(1, timeout=10000)

await run_pw(_step)

## 「KAKEN」側候補をクリックする

「データ管理者の e-Rad 研究者番号」「データ管理者 (日本語)」「Data manager (English)」「データ管理者の所属組織名 (日本語)」が入力されること。 「Contact organization of data manager (English)」は空欄であること


In [ ]:
async def _step(page):
    await page.locator('(//*[contains(text(), "50735975")])[1]').click()
    
    await expect(page.locator('//label[contains(text(), "データ管理者の e-Rad 研究者番号")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "データ管理者の所属組織名 (日本語)")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "Contact organization of data manager (English)")]/../following-sibling::div[1]//input')).not_to_have_value(re.compile('.+'), timeout=1000)
    await expect(page.locator('//label[contains(text(), "データ管理機関コード")]/../following-sibling::div[1]//input')).to_have_value('')
    await expect(page.locator('//label[contains(text(), "データ管理者 (日本語)")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'))
    await expect(page.locator('//label[contains(text(), "Data manager (English")]/../following-sibling::div[1]//input')).to_have_value(re.compile('.+'))

await run_pw(_step)

## 「保存」をクリックする

ダイアログが非表示になること

In [ ]:
async def _step(page):
    await page.locator('//a[text() = "保存"]').click()
    await expect(page.locator('//label[contains(text(), "メタデータのアクセス権")]/../following-sibling::select[1]')).not_to_be_visible(timeout=10000)

await run_pw(_step)

後始末

In [ ]:
await finish_pw_context()

In [ ]:
!rm -fr {work_dir}